In [5]:
import numpy as np
import pandas as pd
import hicstraw
from matplotlib import pyplot as plt

import operator 
""" 
    for flipping the True and False value in list.
    https://bobbyhadz.com/blog/python-negate-boolean 
"""

' \n    for flipping the True and False value in list.\n    https://bobbyhadz.com/blog/python-negate-boolean \n'

In [6]:
hic_path = "/media/jordan990301/Samsung_T5/HiC_Datasets/Rao_2014/GM12878/MAPQGE30/GSE63525_GM12878_insitu_primary_replicate_combined_30.hic"
input_path = "/home/jordan990301/PCA_Experiments/outputs/GM12878/100Kb/origin"
output_plot_path = "/home/jordan990301/PCA_Experiments/outputs/plots/EV1_VS_CorrEntry1/100Kb"
output_log_path = "/home/jordan990301/PCA_Experiments/outputs/Logs/EV1_VS_CorrEntry1/100Kb"

In [7]:
hic = hicstraw.HiCFile(hic_path)
chrom_list = []

for chrom in hic.getChromosomes():
    if (chrom.name != "All" and chrom.name != "MT"):
        chrom_list.append(chrom.name)

print(chrom_list)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']


In [8]:
for chrom in chrom_list:
    # Read in the Pearson correlatin matrix
    pearson_df = pd.read_table(f"{input_path}/origin_pearson_chrom{chrom}.txt", header=None, sep=" ")
    pearson_df.pop(pearson_df.columns[-1])
    pearson_df = pearson_df.dropna(axis=0, how="all").reset_index(drop=True)
    pearson_df = pearson_df.dropna(axis=1, how="all")
    # These code are required since dropping the columns in pandas can't be renamed to index directly. 
    numpy_pearson_df = pearson_df.values
    pearson_df = pd.DataFrame(numpy_pearson_df)
    pearson_df = pd.DataFrame(pearson_df[0])

    # Read in the Eigenvector 1
    EV1_df = pd.read_table(f"{input_path}/origin_pc1_chrom{chrom}.txt", header=None, sep=" ")
    EV1_df = EV1_df.dropna(axis=0, how="all").reset_index(drop=True)

    # Compare the difference between the sign of these two signals.
    Pearson_Entry1_Pos = pearson_df > 0
    EV1_Pos = EV1_df > 0
    
    # Turn into python list
    Pearson_Entry1_Pos = list(Pearson_Entry1_Pos[0].values)
    EV1_Pos = list(EV1_Pos[0].values)
    
    Difference_Count = 0
    for i in range(len(Pearson_Entry1_Pos)):
        if(Pearson_Entry1_Pos[i] != EV1_Pos[i]):
            Difference_Count += 1

    with open(f'{output_log_path}/GM12878_Chr{chrom}_EV1_CorrEntry1_diff.txt',"w+") as f:
        f.write("Pearson_Entry1_Pos entry num: ")
        f.write(str(len(Pearson_Entry1_Pos)))

        f.write("\n")
        f.write("EV1_Pos entry num: ")
        f.write(str(len(EV1_Pos)))

        f.write("\n")
        f.write("Difference_Count: ")
        f.write(str(Difference_Count))
    
    """_summary_
        The operator.not_ method is mostly used when you need to flip a list of boolean values with map().
        https://bobbyhadz.com/blog/python-negate-boolean 
        Since the sign of EV1 were arbitrary asigned, sometimes we need to flip the EV1 to match the result of CorrEntry1.
    """
    if Difference_Count > 0.5 * len(Pearson_Entry1_Pos):
        EV1_Pos = list(map(operator.not_, EV1_Pos))

    # Visualization
    EV1_Viz = []
    Pearson_Entry1_Viz = []
    plt.xticks(np.arange(0, len(Pearson_Entry1_Pos), 50)) 
    plt.rcParams["figure.figsize"] = [20, 10]
    plt.rcParams["figure.autolayout"] = True

    for i in EV1_Pos:
        if i:
            EV1_Viz.append(1)
        else:
            EV1_Viz.append(-1)

    plt.plot(EV1_Viz, color="blue", label="EV1")

    for i in Pearson_Entry1_Pos:
        if i:
            Pearson_Entry1_Viz.append(0.5)
        else:
            Pearson_Entry1_Viz.append(-0.5)

    plt.plot(Pearson_Entry1_Viz, color="red", label="CorrEntry1")
    
    plt.legend(loc="upper left")
    plt.savefig(f'{output_plot_path}/GM12878_Chr{chrom}_100Kb.png')
    
    # Reset the plt configure.
    plt.clf()

<Figure size 2000x1000 with 0 Axes>